In [40]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import Dict, designs, MetalGUI

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
import numpy as np
from collections import OrderedDict
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital


In [42]:
design = designs.DesignPlanar(overwrite_enabled=True)

In [43]:
gui = MetalGUI(design)

In [44]:
design.variables['cpw_width']='20um'
design.variables['cpw_gap']='12um'

# 3 ressonadores com acoplamento capacitivo

In [72]:
design.delete_all_components()

Logger issue: Internal C++ object (QTextEditLogger) already deleted.


In [79]:
cap_opt= Dict(
    pos_x='-4mm',
    pos_y='0mm',
    orientation='330',
    north_width='20um',
    north_gap='12um',
    south_width='20um',
    south_gap='12um',
    cap_width='5um',
    cap_gap='6um',
    cap_gap_ground='12um',
    finger_length='55um',
    finger_count='3',
    cap_distance='1um'
)
cap1 = CapNInterdigital(design, 'CAP1', options = cap_opt)

cap_opt.pos_x = '4mm'
cap_opt.orientation = '390'
cap2 = CapNInterdigital(design, 'CAP2', options=cap_opt)

cap_opt.pos_x = '0mm'
cap_opt.pos_y = '-7mm'
cap_opt.orientation='90'
cap3 = CapNInterdigital(design, 'CAP3', options=cap_opt)

res1 = RouteStraight(design, 'RES1', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'CAP1',
            pin = 'north_end'
        ),
        end_pin = Dict(
            component = 'CAP2',
            pin = 'north_end'
        )
    ),
    lead = Dict(start_straight = '500um', end_straight = '500um'),
    fillet='200um',
    hfss_wire_bonds = True
))

res2 = RouteStraight(design, 'RES2', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'CAP1',
            pin = 'south_end'
        ),
        end_pin = Dict(
            component = 'CAP3',
            pin = 'north_end'
        )
    ),
    lead = Dict(start_straight = '470um', end_straight = '470um'),
    fillet='200um',
    hfss_wire_bonds = True
))

res3 = RouteStraight(design, 'RES3', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'CAP2',
            pin = 'south_end'
        ),
        end_pin = Dict(
            component = 'CAP3',
            pin = 'south_end'
        )
    ),
    lead = Dict(start_straight = '510um', end_straight = '510um'),
    fillet='200um',
    hfss_wire_bonds = True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-3mm',
    pos_y='1.5mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='3mm',
    pos_y='1.5mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

from collections import OrderedDict
anchors = OrderedDict()
anchors[0] = np.array([-0.425,1])
anchors[1] = np.array([0,0.500])
anchors[2] = np.array([0.425,1])

line1 = RouteAnchors(design, 'LINE1', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'PAD1',
            pin = 'tie'
        ),
        end_pin = Dict(
            component = 'PAD2',
            pin = 'tie'
        )
    ),
    anchors = anchors,
    lead=Dict(start_straight='1mm', end_straight='1mm'),
    total_length='8.5mm',
    fillet='200um',
    hfss_wire_bonds = False
))

otg1 = OpenToGround(design, 'OTG1', options=Dict(
    pos_x='2.662mm',
    pos_y='-3.5mm',
    orientation='60',
    width='20um',
    gap='12um',
    termination_gap='12um'
))

pad3 = LaunchpadWirebond(design, 'PAD3', options=Dict(
    pos_x='3mm',
    pos_y='-7.5mm',
    orientation='90',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

line2 = RouteAnchors(design, 'LINE2', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'OTG1',
            pin = 'open'
        ),
        end_pin = Dict(
            component = 'PAD3',
            pin = 'tie'
        )
    ),
    lead=Dict(start_straight='830um', end_straight='1mm'),
    fillet='200um',
    hfss_wire_bonds = False
))

gui.rebuild()
gui.autoscale()

# 3 ressonadores com acoplamento indutivo

In [107]:
design.delete_all_components()

In [108]:
h = 6.89
otg1 = OpenToGround(design, 'OTG1', options=Dict(
    pos_x='-4.16mm',
    pos_y=f'{h}mm',
    orientation='180',
    width='20um',
    gap='12um',
    termination_gap='12um'
))

otg2 = OpenToGround(design, 'OTG2', options=Dict(
    pos_x='4.16mm',
    pos_y=f'{h}mm',
    orientation='0',
    width='20um',
    gap='12um',
    termination_gap='12um'
))

otg3 = OpenToGround(design, 'OTG3', options=Dict(
    pos_x='-4.16mm',
    pos_y=f'{h-0.07}mm',
    orientation='180',
    width='20um',
    gap='12um',
    termination_gap='12um'
))

otg4 = OpenToGround(design, 'OTG4', options=Dict(
    pos_x='4.16mm',
    pos_y=f'{h-0.07}mm',
    orientation='0',
    width='20um',
    gap='12um',
    termination_gap='12um'
))

otg5 = OpenToGround(design, 'OTG5', options=Dict(
    pos_x='-0.035mm',
    pos_y='0mm',
    orientation='-90',
    width='20um',
    gap='12um',
    termination_gap='12um'
))

otg6 = OpenToGround(design, 'OTG6', options=Dict(
    pos_x='0.035mm',
    pos_y='0mm',
    orientation='-90',
    width='20um',
    gap='12um',
    termination_gap='12um'
))

res1 = RouteStraight(design, 'RES1', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'OTG1',
            pin = 'open'
        ),
        end_pin = Dict(
            component = 'OTG2',
            pin = 'open'
        )
    ),
    fillet='200um',
    hfss_wire_bonds = True
))

res2 = RouteStraight(design, 'RES2', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'OTG3',
            pin = 'open'
        ),
        end_pin = Dict(
            component = 'OTG5',
            pin = 'open'
        )
    ),
    lead = Dict(start_straight = '830um', end_straight = '830um'),
    fillet='200um',
    hfss_wire_bonds = True
))

res3 = RouteStraight(design, 'RES3', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'OTG4',
            pin = 'open'
        ),
        end_pin = Dict(
            component = 'OTG6',
            pin = 'open'
        )
    ),
    lead = Dict(start_straight = '830um', end_straight = '830um'),
    fillet='200um',
    hfss_wire_bonds = True
))

pad1 = LaunchpadWirebond(design, 'PAD1', options=Dict(
    pos_x='-3mm',
    pos_y='8mm',
    orientation='0',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

pad2 = LaunchpadWirebond(design, 'PAD2', options=Dict(
    pos_x='3mm',
    pos_y='8mm',
    orientation='180',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

from collections import OrderedDict
anchors = OrderedDict()
anchors[0] = np.array([-0.430,7.5])
anchors[1] = np.array([0,6.96])
anchors[2] = np.array([0.430,7.5])

line1 = RouteAnchors(design, 'LINE1', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'PAD1',
            pin = 'tie'
        ),
        end_pin = Dict(
            component = 'PAD2',
            pin = 'tie'
        )
    ),
    anchors = anchors,
    lead=Dict(start_straight='1mm', end_straight='1mm'),
    total_length='8.5mm',
    fillet='200um',
    hfss_wire_bonds = False
))

otg7 = OpenToGround(design, 'OTG7', options=Dict(
    pos_x='2.133mm',
    pos_y='4.5mm',
    orientation='60',
    width='20um',
    gap='12um',
    termination_gap='12um'
))

pad3 = LaunchpadWirebond(design, 'PAD3', options=Dict(
    pos_x='3mm',
    pos_y='1mm',
    orientation='90',
    lead_length='25um',
    pad_width='350um',
    pad_height='350um',
    pad_gap='150um',
    taper_height='370um'
))

line2 = RouteAnchors(design, 'LINE2', options=Dict(
    pin_inputs = Dict(
        start_pin = Dict(
            component = 'OTG7',
            pin = 'open'
        ),
        end_pin = Dict(
            component = 'PAD3',
            pin = 'tie'
        )
    ),
    lead=Dict(start_straight='830um', end_straight='1mm'),
    fillet='200um',
    hfss_wire_bonds = False
))

gui.rebuild()
gui.autoscale()
#gui.zoom_on_components(['OTG7'])

# Q3D

In [13]:
from qiskit_metal import renderers

In [14]:
ansys = design.renderers.q3d

In [16]:
ansys.connect_ansys()

INFO 10:36AM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:36AM [load_ansys_project]: 	Opened Ansys App
INFO 10:36AM [load_ansys_project]: 	Opened Ansys Desktop v2025.1.0
INFO 10:36AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/e196203/Documents/Ansoft/
	Project:   Sub redes indutivas
INFO 10:36AM [connect_design]: 	Opened active design
	Design:    Capacitancias sub rede 3 capacitiva [Solution type: Q3D]
INFO 10:36AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 10:36AM [connect]: 	Connected to project "Sub redes indutivas" and design "Capacitancias sub rede 3 capacitiva" 😀 



In [23]:
ansys.activate_q3d_design('Capacitancias sub rede 3 indutiva')

01:06PM 48s WARNING [activate_q3d_design]: This method is deprecated. Change your scripts to use activate_ansys_design()
INFO 01:06PM [connect_design]: 	Opened active design
	Design:    Capacitancias sub rede 3 indutiva [Solution type: Q3D]


In [24]:
ansys.clean_active_design()
ansys.render_design()

In [19]:
ansys.add_q3d_setup(
    name='capacitancias',
    freq_ghz=7,
    min_passes=2,
    max_passes=15,
    percent_error=0.2
)

In [25]:
ansys.analyze_setup('capacitancias')

INFO 01:09PM [get_setup]: 	Opened setup `capacitancias`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 01:09PM [analyze]: Analyzing setup capacitancias


In [26]:
ansys.get_capacitance_matrix()

INFO 01:22PM [get_matrix]: Exporting matrix data to (D:\Users\e196203\AppData\Local\Temp\2\tmpjqqswh5o.txt, C, , capacitancias:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 7000000000, Maxwell, 1, False


(                       g_wb  launch_pad_PAD1  trace_RES1  trace_RES2  \
 g_wb             7397.19019      -2763.60296 -1358.05300 -1389.95836   
 launch_pad_PAD1 -2763.60296       2775.97885    -5.91180    -0.03046   
 trace_RES1      -1358.05300         -5.91180  1377.68069    -6.58416   
 trace_RES2      -1389.95836         -0.03046    -6.58416  1404.29181   
 trace_RES3      -1389.42717         -0.02913    -6.58240    -6.85900   
 
                  trace_RES3  
 g_wb            -1389.42717  
 launch_pad_PAD1    -0.02913  
 trace_RES1         -6.58240  
 trace_RES2         -6.85900  
 trace_RES3       1403.73959  ,
 'fF')

# Eigenmode

In [27]:
ansys.disconnect_ansys()

In [28]:
ansys = design.renderers.hfss

In [29]:
ansys.connect_ansys()

INFO 01:26PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:26PM [load_ansys_project]: 	Opened Ansys App
INFO 01:26PM [load_ansys_project]: 	Opened Ansys Desktop v2025.1.0
INFO 01:26PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/e196203/Documents/Ansoft/
	Project:   Sub redes indutivas
INFO 01:26PM [connect_design]: 	Opened active design
	Design:    Capacitancias sub rede 3 indutiva [Solution type: Q3D]
INFO 01:26PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 01:26PM [connect]: 	Connected to project "Sub redes indutivas" and design "Capacitancias sub rede 3 indutiva" 😀 



In [30]:
ansys.activate_eigenmode_design('sub rede 3 indutiva')

01:26PM 53s WARNING [activate_eigenmode_design]: This method is deprecated. Change your scripts to use activate_ansys_design()
01:26PM 53s WARNING [activate_ansys_design]: The design_name=sub rede 3 indutiva was not in active project.  Designs in active project are: 
['Capacitancias sub rede 3 capacitiva', 'Capacitancias sub rede 3 indutiva', 'sub rede 3 capacitiva', 'S parameter sub rede 3 capacitivo'].  A new design will be added to the project.  
INFO 01:27PM [connect_design]: 	Opened active design
	Design:    sub rede 3 indutiva [Solution type: Eigenmode]
WARNING 01:27PM [connect_setup]: 	No design setup detected.
WARNING 01:27PM [connect_setup]: 	Creating eigenmode default setup.
INFO 01:27PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [31]:
ansys.clean_active_design()
ansys.render_design()

In [32]:
ansys.add_eigenmode_setup(
    name = 'ressonancias',
    n_modes = 4,
    min_passes = 15,
    max_passes = 20,
    min_converged = 2
)

In [33]:
ansys.activate_eigenmode_setup('ressonancias')

02:11PM 40s WARNING [activate_eigenmode_setup]: This method is deprecated. Change your scripts to use activate_ansys_setup()
INFO 02:11PM [get_setup]: 	Opened setup `ressonancias`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [34]:
ansys.analyze_setup('ressonancias')

INFO 02:11PM [get_setup]: 	Opened setup `ressonancias`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 02:11PM [analyze]: Analyzing setup ressonancias


02:28PM 42s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Não há cota suficiente para processar este comando.)

02:28PM 43s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Não há cota suficiente para processar este comando.)



In [36]:
ansys.pinfo.setup.get_solutions().eigenmodes()

([3.51542343479, 6.91708717802, 6.98210806445, 7.04488899843], None)

In [ ]:
ansys.plot_ansys_fields('main')

# Drivenmode

In [100]:
ansys = design.renderers.hfss

In [101]:
ansys.connect_ansys()

INFO 05:31PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:31PM [load_ansys_project]: 	Opened Ansys App
INFO 05:31PM [load_ansys_project]: 	Opened Ansys Desktop v2025.1.0
INFO 05:31PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/e196203/Documents/Ansoft/
	Project:   Sub redes indutivas
INFO 05:31PM [connect_design]: 	Opened active design
	Design:    sub rede 3 indutiva [Solution type: Eigenmode]
INFO 05:31PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 05:31PM [connect]: 	Connected to project "Sub redes indutivas" and design "sub rede 3 indutiva" 😀 



In [103]:
ansys.activate_drivenmodal_design('S parameter sub rede 3 indutivo')

05:32PM 11s WARNING [activate_drivenmodal_design]: This method is deprecated. Change your scripts to use activate_ansys_design()
INFO 05:32PM [connect_design]: 	Opened active design
	Design:    S parameter sub rede 3 indutivo [Solution type: HFSS Hybrid Modal Network]
05:32PM 12s WARNING [activate_ansys_design]: The design_name=S parameter sub rede 3 indutivo already exists, but it has solution_type==hfss hybrid modal network, which is different from the requested==drivenmodal. If you want a design with solution type==drivenmodal, please change the name requested for your design to one that does not exist. Alternatively, manually modify the solution_type for design S parameter sub rede 3 indutivo from the Ansys GUI.


In [110]:
ansys.clean_active_design()
ansys.render_design(
    selection=['RES1','RES2','RES3','LINE1','LINE2'],
    open_pins=[('LINE1','end'),('RES1','start'),('RES1','end'),('RES2','start'),('RES2','end'),('RES3','start'),('RES3','end')],
    port_list=[('LINE1','start',50),('LINE2','end',50)]
)

In [111]:
ansys.add_drivenmodal_setup(
    name='Setup',
    min_passes=22,
    max_passes=25,
    min_converged=5,
    max_delta_s=0.002,
    basis_order=-1
)

In [112]:
gui.main_window.close()

True